## Market Basket Analysis - Groceries Dataset

In [ ]:
Author: Khushee Kapoor

Last Updated: 20/12/2021

**Meta-Data** of the Groceries Dataset:

The dataset contains a list of transactions and the items purchased in each transaction in a Super-Market.

First, we import the libraries:

*   NumPy - for data manipulation.
*   Pandas - for data manipulation.
*   MatPlotLib - for data visualization.
*   Seaborn - for data visualization.
*   CSV Reader - to read the dataset.
*   MLXtend - to apply the Apriori Algorithm.

In [ ]:
# importing the libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from csv import reader
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

Next, we read the dataset into a list. Here, we use the csv_reader() function fromt the CSV Reader library instead of the read_csv() function from the Pandas library because the number of columns in each row is different (ragged arrays).

In [ ]:
# reading the dataset
groceries = []
with open('../input/groceries/groceries.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        groceries.append(row)

To solve the following questions, we prepare the transaction dataframe to a format to which we can apply the Apriori algorithm.

Naive approach to form the transaction dataframe:

1.   To find the unique items - flatten the list and convert it into a set. The conversion removes any duplicate values, and hence, we are left with only the unique items in the dataset.
2.   Convert the set of unique items into an empty Pandas dataframe.
3.   Find every item in a transaction and append 1 if found and 0 if not found. This fills the empty dataframe previously created.



In [ ]:
# items = set(sum(groceries, []))

In [ ]:
# df = pd.DataFrame(columns=items)

In [ ]:
# for i in range(len(groceries)):
#     transaction = []
#     for item in items:
#         if item in groceries[i]:
#             transaction.append(1)
#         else:
#             transaction.append(0)
#     print(transaction)
#     df = df.append(transaction, ignore_index=True)          

The above method works, but is very time inefficient. To solve the problem in significantly less time, we use the TransactionEncoder class from the MLXtend library. We fit the object of the class on the list and convert it into a Pandas dataframe with 1 representing item is purchased in that transaction, and 0 representing that the item is not purchased in that transaction.

In [ ]:
# fitting the list and converting the transactions to true and false
encoder = TransactionEncoder()
transactions = encoder.fit(groceries).transform(groceries)

In [ ]:
# converting the true and false to 1 and 0
transactions = transactions.astype('int')

In [ ]:
# converting the transactions array to a datafrmae
df = pd.DataFrame(transactions, columns=encoder.columns_)

In [ ]:
# viewing the first few rows of the dataframe
df.head()

#### **Q1** How many transactions and items are there in the data set?


To solve Question 1, we use the shape attribute of a Pandas dataframe. Here, the number of rows represent the number of transactions, and the number of columns represent the number of items in the dataset.

In [ ]:
# finding the dimensions of the dataframe
df.shape

As we can see, there are 9835 rows, meaning 9835 transactions, and 169 columns, meaning 169 items in the dataset.

To prepare the data for the following questions, we apply the Apriori algorithm on the dataframe and set the minimum support parameter to 2%.

In [ ]:
# applying the apriori algorithm
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

#### **Q4** Find top selling items with minimum support of 2%.

To solve Question 4, first we sort the dataframe by support in the descending order by using the sort_values() function from the Pandas library and setting the by and ascending parameters to support and False respectively.

In [ ]:
# sorting the dataframe
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

Next, we filter the dataframe to find items with length 1 and support more than equal to 2%. Then we splice the sorted and filtered dataframe to show only the top 5 entries.

In [ ]:
# finding top 5 items with minimum support of 2%
frequent_itemsets[ (frequent_itemsets['length'] == 1) &
                   (frequent_itemsets['support'] >= 0.02) ][0:5]

As we can see, whole milk, other vegetables, rolls/buns, soda and yogurt are the top 5 selling items with support of 25%, 19%, 18%, 17%, and 14% respectively.

#### **Q5.** Find all frequent itemsets with minimum support of 5%.

To solve Question 5, we filter the dataframe to find itemsets having length more than 1, and support more than 5%.

In [ ]:
# finding itemsets having length more than 1 and minimum support of 5%
frequent_itemsets[(frequent_itemsets['length'] > 1) & 
                  (frequent_itemsets['support'] >= 0.05)]

As we can see, there are only 3 itemsets - other vegetables and whole milk, rolls/buns and whole milk, and yogurt and whole milk each of length 2 and having support of 7%, 5.66% and 5.60% respectively.

#### **Q6.**  Find all frequent itemsets of length 2 with minimum support of 2%.

To solve Question 6, we filter the dataframe to find itemsets having length 2 and minimum support of 2%.

In [ ]:
# finding itemsets having length 2 and minimum support of 2%
frequent_itemsets[(frequent_itemsets['length'] == 2) & 
                  (frequent_itemsets['support'] >= 0.02)]

As we can see, there are 61 itemsets having length 2 with support more than or equal to 2%. The support ranges between 7% and 2% with other vegetables and whole milk having the highest support, and butter and other vegetables having the minimum support.

#### **Q7.** Find the top 10 association rules with minimum support of 2%, sorted by confidence in descending order.


To solve Question 7, we first find the association rules using the association_rules() function from the MLXtend library and set the parameter metric to support, and the min_threshold to 2%.

In [ ]:
# finding top 10 association rules with minimum support of 2%
rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.02)
rules

Then we sort the generated association rules in the descending order by confidence by using the sort_values() function from the Pandas library and setting the by and ascending parameters to confidence and False respectively. Then we splice the sorted dataframe to show the top 10 rules.

In [ ]:
# sorting the rules in the descending order by confidence
rules.sort_values(by='confidence', ascending=False)[0:10]

As we can see, the top association rule is that if a customer buys other vegetables and yogurt, they also by whole milk, with a support of 2%, confidence of 51% and lift value of 2 indicating a positive correlation (if the sales of other vegetables and yogurt goes up, the sales of whole milk also goes up and vice versa). Similarly, we can read and interpret the other rules.

#### **Q8.** Find association rules with minimum support of 2% and lift of more than 1.0.


To solve Question 8, we filter the dataframe to have lift more than 1.

In [ ]:
# finding association rules with minimum support of 2% and having lift more than 1
rules[(rules['support'] >= 0.02) &
      (rules['lift'] > 1.0)]

As we can see, there are 126 rules having support of 2% or more and lift more than 1. All the items in these rules have a positive correlation with each other, indicating if the sales of one goes up, the sales of the other goes up as well and vice versa.